In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np 

In [ ]:
# Есть сервис доставки еды
# Есть табличка:
#order_id - айди заказа
#confirmed_at — время оплаты
#delivered_at — время доставки

#Нужно написать SQL-запрос, который поможет найти пиковые часы, в которые не хватает курьеров.

In [2]:
day_counter = pd.to_datetime('2022-01-01') # Первая дата заказа
end_date = pd.to_datetime('2022-01-31') # Последняя дата заказа
order_id_offset = 1 # Счетчик id заказов

In [6]:
# Генерация датафрейма заказов от 1го до 31го января

orders = pd.DataFrame() # Объявляю основной датафрейм
while day_counter <= end_date:
    daily_orders_count = np.random.randint(80,120) # Случайное колличество заказов в день
    
    order_ids = np.arange(order_id_offset, order_id_offset+daily_orders_count) # Массив ID заказов с учетом притока
    times = pd.date_range(start = day_counter, periods=96, freq='15min') # Массив возможных значений времени заказа
    
    confirmed_at = np.random.choice(times, daily_orders_count) # Выбираем случайное время заказа
    
    day_user_purchases = pd.DataFrame({'order_id': order_ids, 
                             'confirmed_at': np.sort(confirmed_at),
                             'delivered_at': np.sort(confirmed_at)}) #Собираем заказы за день

    #Смещаем каждую дату дату заказа на случайное время доставки от 15 до 90 минут
    for ind in day_user_purchases.index:
        day_user_purchases['delivered_at'][ind] = day_user_purchases['confirmed_at'][ind] + pd.DateOffset(minutes = np.random.randint(15,90))

    
    orders = orders.append(day_user_purchases, ignore_index=True) # Наполняем основнй датафрейм

  
    order_id_offset += daily_orders_count #Переходим к следующей пачке заказов
    day_counter = day_counter + pd.DateOffset(1) # Переходим к следующему дню

/tmp/ipykernel_939587/2966727726.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_user_purchases['delivered_at'][ind] = day_user_purchases['confirmed_at'][ind] + pd.DateOffset(minutes = np.random.randint(15,90))


In [8]:
orders

,order_id,confirmed_at,delivered_at
0,1,2022-01-01 00:00:00,2022-01-01 01:24:00
1,2,2022-01-01 00:00:00,2022-01-01 01:23:00
2,3,2022-01-01 00:45:00,2022-01-01 01:26:00
3,4,2022-01-01 00:45:00,2022-01-01 01:45:00
4,5,2022-01-01 02:00:00,2022-01-01 03:00:00
...,...,...,...
3019,3020,2022-01-31 23:15:00,2022-02-01 00:25:00
3020,3021,2022-01-31 23:15:00,2022-02-01 00:20:00
3021,3022,2022-01-31 23:15:00,2022-02-01 00:34:00
3022,3023,2022-01-31 23:30:00,2022-02-01 00:55:00


In [22]:
# ТОП-5 загруженных часа.
q = """
SELECT * 
FROM (    
    SELECT
    strftime('%H',confirmed_at) as hour_of_day,
    avg(Cast((JulianDay(delivered_at) - JulianDay(confirmed_at)) * 24 * 60 As Integer)) as avg_delivery_time
    FROM orders
    GROUP BY strftime('%H',confirmed_at)
    ORDER BY avg_delivery_time desc)
LIMIT 5;"""
sqldf(q)

,hour_of_day,avg_delivery_time
0,13,55.204545
1,20,54.043165
2,16,53.728000
3,03,53.477941
4,10,52.903448


In [19]:
# Просто посмотреть среднее время доставки по часам дня
q = """
SELECT
strftime('%H',confirmed_at) as hour_of_day,
avg(Cast((JulianDay(delivered_at) - JulianDay(confirmed_at)) * 24 * 60 As Integer)) as avg_delivery_time
FROM orders
GROUP BY strftime('%H',confirmed_at);"""
sqldf(q)

,hour_of_day,avg_delivery_time
0,00,51.331034
1,01,50.051282
2,02,48.645669
3,03,53.477941
4,04,49.946565
5,05,49.929204
6,06,49.888112
7,07,49.790323
8,08,47.133929
9,09,52.519608
